# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Set Hyper Parameters

In [3]:
nepochs = 100
batch_size = 1024
learning_rate = 0.01

window_size = 100

# Set Path

In [4]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")

# Help Functions

In [5]:
def calc_haversine(lat1, lon1, lat2, lon2):
    
    lat1=lat1 % 360
    lon1=lon1 % 360
    lat2=lat2 % 360
    lon2=lon2 % 360
    
    dlat = (lat2 - lat1) % 360
    dlon = (lon2 - lon1) % 360 
    
    dlat, dlon = map(np.radians, [dlat, dlon])
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(a ** 0.5)
        
    dist = 6_367_000 * c
    return dist

In [6]:
def check_score_np(predict:torch.Tensor, target:torch.Tensor):
    m = []
    predict = predict.detach().numpy()
    target = target.detach().numpy()
    for i in range(predict.shape[0]):
        temp = calc_haversine(predict[i,0], predict[i,1], target[i,0], target[i,1])
        m.append(temp)
    
    m = np.array(m)
    m = m[np.invert(np.isnan(m))]
    if m.shape[0] == 0:
        score = np.inf
    else:
        score = (np.percentile(m, 50) + np.percentile(m, 95))/2
    
    return score

# Load Data

In [7]:
df_train_default = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))

In [8]:
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [9]:
for col in df_train_default.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
TimeNanos
LeapSecond
FullBiasNanos
BiasNanos
BiasUncertaintyNanos
DriftNanosPerSecond
DriftUncertaintyNanosPerSecond
HardwareClockDiscontinuityCount
Svid
TimeOffsetNanos
State
ReceivedSvTimeNanos
ReceivedSvTimeUncertaintyNanos
Cn0DbHz
PseudorangeRateMetersPerSecond
PseudorangeRateUncertaintyMetersPerSecond
AccumulatedDeltaRangeState
AccumulatedDeltaRangeMeters
AccumulatedDeltaRangeUncertaintyMeters
CarrierFrequencyHz
MultipathIndicator
ConstellationType
AgcDb
BasebandCn0DbHz
FullInterSignalBiasNanos
FullInterSignalBiasUncertaintyNanos
SatelliteInterSignalBiasNanos
SatelliteInterSignalBiasUnc

# Dataloader

In [10]:
df_train_default['phone'].value_counts()

2021-04-22-US-SJC-1_Pixel4             2890
2021-04-22-US-SJC-1_SamsungS20Ultra    2826
2020-09-04-US-SF-2_Mi8                 2500
2021-04-29-US-SJC-2_SamsungS20Ultra    2370
2020-09-04-US-SF-2_Pixel4              2349
                                       ... 
2021-01-05-US-SVL-2_Pixel4XL           1193
2020-06-05-US-MTV-1_Pixel4XLModded     1123
2021-04-26-US-SVL-1_Mi8                1036
2021-04-26-US-SVL-1_Pixel5             1034
2020-05-14-US-MTV-2_Pixel4XLModded      577
Name: phone, Length: 73, dtype: int64

In [11]:
def CustomTrainValidSplit(df:pd.DataFrame, valid_size):
    phones = df['phone'].unique()
    
    valid_num = int(len(phones) * valid_size)
    train_num = len(phones) - valid_num
    
    indexes = np.array(range(len(phones)))
    indexes = np.random.choice(indexes, len(indexes))
    
    df_train = []
    for phone in phones[indexes[:train_num]]:
        df_train.append(df[df['phone'] == phone])
    df_train = pd.concat(df_train)
    
    df_valid = []
    for phone in phones[indexes[train_num:-1]]:
        df_valid.append(df[df['phone'] == phone])
    df_valid = pd.concat(df_valid)
    
    return df_train, df_valid
    
df_train, df_valid = CustomTrainValidSplit(df_train_default, valid_size = 0.1)
print(df_train.shape, df_valid.shape)
    

(119059, 108) (9836, 108)


In [12]:
df_train

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,...,BiasYMicroT,BiasZMicroT,utcTimeMillis_OrientationDeg,elapsedRealtimeNanos_OrientationDeg,yawDeg,rollDeg,pitchDeg,dlatDeg,dlngDeg,dheight
77129,2021-01-04-US-RWC-2,Pixel4Modded,1293835203744,37.444346,-122.232676,21.33,2021-01-04-US-RWC-2_Pixel4Modded,235.74,0.8,0.0,...,-3.95323,9.785788,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.00
77133,2021-01-04-US-RWC-2,Pixel4Modded,1293835204733,37.444302,-122.232650,27.07,2021-01-04-US-RWC-2_Pixel4Modded,236.73,0.8,0.0,...,-3.95323,9.785788,NaN,NaN,NaN,NaN,NaN,-0.000044,0.000026,5.74
77137,2021-01-04-US-RWC-2,Pixel4Modded,1293835205744,37.444356,-122.232629,16.75,2021-01-04-US-RWC-2_Pixel4Modded,237.74,0.8,0.0,...,-3.95323,9.785788,NaN,NaN,NaN,NaN,NaN,0.000010,0.000047,-4.58
77141,2021-01-04-US-RWC-2,Pixel4Modded,1293835206741,37.444386,-122.232616,22.32,2021-01-04-US-RWC-2_Pixel4Modded,238.74,0.8,0.0,...,-3.95323,9.785788,NaN,NaN,NaN,NaN,NaN,0.000040,0.000060,0.99
77145,2021-01-04-US-RWC-2,Pixel4Modded,1293835207732,37.444353,-122.232636,15.18,2021-01-04-US-RWC-2_Pixel4Modded,239.73,0.8,0.0,...,-3.95323,9.785788,NaN,NaN,NaN,NaN,NaN,0.000007,0.000040,-6.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131337,2021-04-29-US-MTV-1,SamsungS20Ultra,1303770559000,37.395790,-122.103020,21.70,2021-04-29-US-MTV-1_SamsungS20Ultra,1758.00,0.9,0.0,...,-33.96000,46.320000,1.619735e+12,7.922927e+13,180.0,-180.0,0.0,-0.000003,0.000008,23.00
131338,2021-04-29-US-MTV-1,SamsungS20Ultra,1303770560000,37.395784,-122.103041,21.39,2021-04-29-US-MTV-1_SamsungS20Ultra,1759.00,0.9,0.0,...,-33.96000,46.320000,1.619735e+12,7.923027e+13,180.0,-180.0,0.0,-0.000008,-0.000013,22.69
131339,2021-04-29-US-MTV-1,SamsungS20Ultra,1303770561000,37.395813,-122.103052,9.87,2021-04-29-US-MTV-1_SamsungS20Ultra,1760.00,0.9,0.0,...,-33.96000,46.320000,1.619735e+12,7.923127e+13,180.0,-180.0,0.0,0.000021,-0.000024,11.17
131340,2021-04-29-US-MTV-1,SamsungS20Ultra,1303770562000,37.395822,-122.103019,10.75,2021-04-29-US-MTV-1_SamsungS20Ultra,1761.00,0.9,0.0,...,-33.96000,46.320000,1.619735e+12,7.923227e+13,180.0,-180.0,0.0,0.000029,0.000009,12.05


In [13]:
def GetWindows(idx, window_size):
    index = np.array([])
    if idx < window_size:
        index = np.concatenate([np.zeros(window_size - idx-1), np.array(range(idx+1))])
        pass
    else:
        index = np.array(range(idx-window_size+1, idx+1))
    return index.astype(int)


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.DataFrame, 
                 features = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM'], 
                 labels = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM'],
                 window_size = 100,
                 train = True,
                device = 'cpu'):
        self.df = df
        self.features = features
        self.labels = labels
        self.len = df.shape[0]
        self.window_size = window_size
        self.train = train
        self.device = device
        
        self.data = self.df[features].astype(float).values
        if train == True:
            self.true = self.df[labels].astype(float).values
        else:
            self.true = []
        self.phone = self.df['phone'].values
        self.millisSinceGpsEpoch = self.df['millisSinceGpsEpoch'].values
        
        self.start_index_by_phone = dict()
        
        for phone in set(self.phone):
            start_index = np.where(self.phone == phone)[0][0]
            self.start_index_by_phone[phone] = start_index
        
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        phone = self.phone[idx]
        start_index = self.start_index_by_phone[phone]
        
        
        window_index = GetWindows(idx - start_index, self.window_size) + start_index
        data = self.data[window_index, :]
        
        if self.train is False:
            true = []
        else:
            true = self.true[idx]
            
        indx = [self.phone[idx], self.millisSinceGpsEpoch[idx]]
        
        # data shape : window_size X num_of_features
        # true shape : num_of_labels X 1
        data = torch.Tensor(data)
        true = torch.Tensor(true.astype(float))
        
        return data, true, idx
    


In [14]:
features = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM',
           'dlatDeg', 'dlngDeg', 'dheight']
labels = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM']

train_data = CustomDataset(df_train, features = features, labels = labels, window_size = window_size, device = device)
valid_data = CustomDataset(df_valid, features = features, labels = labels, window_size = window_size, device = device)
test_data = CustomDataset(df_test, features = features, labels = labels, window_size = window_size, train = False, device = device)

In [15]:
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

# Build Model

In [16]:
torch.deg2rad

<function _VariableFunctionsClass.deg2rad>

In [17]:
def torch_haversine(lat1, lon1, lat2, lon2):
    
    lat1=lat1 % 360
    lon1=lon1 % 360
    lat2=lat2 % 360
    lon2=lon2 % 360
    
    dlat = (lat2 - lat1) % 360
    dlon = (lon2 - lon1) % 360 
    
    dlat, dlon = map(torch.deg2rad, [dlat, dlon])
    a = torch.sin(dlat / 2.0)**2 + torch.cos(lat1) * torch.cos(lat2) * torch.sin(dlon / 2.0)**2
    c = 2 * torch.arcsin(a ** 0.5)
        
    dist = 6_367_000 * c
    return dist

def gps_loss(predict:torch.Tensor, target:torch.Tensor):
    temp = torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])
    
    score = (torch.quantile(m, 0.5) + torch.quantile(m, 0.95))/2
    
    return score

In [18]:
class BaseModel(nn.Module):
    def __init__(self, input_size = (100, 3), output_size = 3):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size[0]*input_size[1], 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 3)
        
    def forward(self, x):
        input_size = self.input_size
        x = x.view(-1, input_size[0]*input_size[1])
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [19]:
model = BaseModel((window_size, len(features)), len(labels))
model.to(device)

# loss_func = nn.SmoothL1Loss()
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                lr_lambda=lambda epoch: 0.995 ** epoch,
                                last_epoch=-1,
                                verbose=True)

Adjusting learning rate of group 0 to 1.0000e-02.


# Fit and Validate

In [20]:
def train(epoch, dataloader):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    prog_bar = tqdm(dataloader, position = 1, desc = 'Train')
    status_log = tqdm(total=0, position = 2, bar_format='{desc}')
    for data, targets, _ in prog_bar:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(outputs)
        ground.append(targets)
        
        status_log.set_description_str(f"training status: [{loss}/{check_score_np(outputs.to('cpu'), targets.to('cpu'))}]")
    scheduler.step()

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = check_score_np(predict.to('cpu'), ground.to('cpu'))
    return loss, meas

In [21]:
def valid(dataloader):
    model.eval()  # 신경망을 추론 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    predict = []
    ground = []
    prog_bar = tqdm(dataloader, position = 1, desc = 'Valid')
    status_log = tqdm(total=0, position = 2, bar_format='{desc}')
    
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets, _ in prog_bar:
            
            data = data.to(device)
            targets = targets.to(device)
            
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
            
            predict.append(outputs)
            ground.append(targets)
            
            status_log.set_description_str(f"valid status: [{loss}/{check_score_np(outputs.to('cpu'), targets.to('cpu'))}]")

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = check_score_np(predict.to('cpu'), ground.to('cpu'))
    return loss, meas

In [22]:
def test(dataloader):
    model.eval()  # 신경망을 추론 모드로 전환
    
    output_list = []
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, _, index in tqdm(dataloader):
            data = data.to(device)
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            output_list.append(outputs)
    
    predicts = torch.cat(output_list)
    predicts = pd.DataFrame(predicts.to('cpu'), columns = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM'])
    return predicts
            

In [ ]:
train_loss_list = []
train_meas_list = []
valid_loss_list = []
valid_meas_list = []

for epoch in range(nepochs):
    train_loss, train_meas = train(epoch, train_loader)
    valid_loss, valid_meas = valid(valid_loader)
    
    train_loss_list.append(train_loss)
    train_meas_list.append(train_meas)
    valid_loss_list.append(valid_loss)
    valid_meas_list.append(valid_meas)
   
    
    print(f"{epoch+1}/{nepochs}: train[{train_loss:.6f}/{train_meas}], valid[{valid_loss}/{valid_meas}]")

history['train_loss'] = train_loss_list
history['train_meas'] = train_meas_list
history['valid_loss'] = valid_loss_list
history['valid_meas'] = valid_meas_list


Train:   0%|          | 0/117 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 9.9500e-03.


Valid:   0%|          | 0/10 [00:00<?, ?it/s]

1/100: train[136720.718750/2478498.813055362], valid[51.92288589477539/203457.23376296027]


Train:   0%|          | 0/117 [00:00<?, ?it/s]

...

Valid:   0%|          | 0/10 [00:00<?, ?it/s]

68/100: train[4.739711/136432.98632077547], valid[4.780490875244141/119185.63156979918]


Train:   0%|          | 0/117 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 7.0761e-03.


Valid:   0%|          | 0/10 [00:00<?, ?it/s]

#  Output

In [ ]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)

In [ ]:
submission.to_csv(f"./models/{notebookName}/{num_files} - result.csv", index = False)
pd.DataFrame([]).to_csv(dummy_path)